In [ ]:
from pyannote.audio import Pipeline
import pickle
import os
import time
import soundfile as sf
from pyannote.core.segment import Segment
import pandas as pd

In [ ]:
def cropp_wav_per_question(audiofile_path, output_folder, start, duration, question, probant_acronym):

    wav_data, wav_samplerate = sf.read(audiofile_path)
    start_sample = int(start * wav_samplerate)  # Assuming you have the sample rate stored in 'samplerate'
    end_sample = start_sample + int(duration * wav_samplerate)
    cropped_data = wav_data[start_sample:end_sample]

    # Save the cropped segment to a new WAV file
    output_file_path = os.path.join(output_folder, f'{probant_acronym}_Interview_{question}.wav')
    sf.write(output_file_path, cropped_data, wav_samplerate)

    print(f"Safed cropped {question} for {probant_acronym}")

In [ ]:
timestamp_data = pd.read_excel('../data/timestamps_manually_corrected.xlsx', sheet_name = 'second_version')
output_folder = '../data/cropped_data/cropped_interview_data/per_question'

for index, row in timestamp_data.iterrows():
    print(row['person'])
    if not (row['speaker_number'].startswith('SPEAKER')):
        continue
    if not (row['Gender'] == 'f'):
        continue
    
    probant_acronym = row['person']
    timepoint = row['time']
    audiofile_folder = f'../data/20221121_Sprachaufnahmen_Preprocessed/{probant_acronym}_Interview_{timepoint}'
    for file_name in os.listdir(audiofile_folder):
        if 'ZOOM' in file_name and not file_name.endswith('BU.WAV') and (file_name.endswith('.WAV') or file_name.endswith('.wav')):
            filename = file_name
            continue
    audiofile_path = os.path.join(audiofile_folder, filename)
    probant_acronym = f'{probant_acronym}_{timepoint}'
    
    for i in [1, 2, 3]:
        
        question = f'Q{i}'
        start = row[f'{question}_Start']
        duration = row[f'{question}_Duration']
            
        cropp_wav_per_question(audiofile_path, output_folder, start, duration, question, probant_acronym)